In [1]:
import numpy as np
import pandas as pd
from climate_indices import indices,compute
from climate_indices.indices import Distribution
from netCDF4 import Dataset
import scipy.stats as st

In [2]:
def fuzzify(x):
  # Add some "measurement error"" to each data point
    for i in range(len(x)):
        if x[i]==0:
            x[i]+=0.005*np.random.uniform(0,1,1)[0]
        else:
            x[i]+=0.005*np.random.uniform(-1,1,1)[0]

In [4]:
f_pre = Dataset('GPCC_half.nc')
data = f_pre.variables['precip']
lon = f_pre.variables['lon'][:]
lat = f_pre.variables['lat'][:]
time = f_pre.variables['time'][:]
data = np.swapaxes(data,0,2)

In [32]:
temporal_limits = {"time_min":datetime(1948, 1, 1, 0, 0),"time_max":datetime(2016, 1, 1, 0, 0) } 

d = Data('GPCC_half.nc','precip',temporal_limits, missing_value=-9.969209968386869e+36)

data = d.get_data()
lon_list = d.get_lon_list()
lat_list = d.get_lat_list()
lon = d.get_lon()
lat = d.get_lat()

In [19]:
r = indices.spi(data[9,0,:],1,Distribution.gamma,1946,1970,2000,compute.Periodicity.monthly)

In [5]:
result = np.zeros(data.shape)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i,j,-1] < 0:
            result[i,j,:] = np.nan
        else:
            fuzzify(data[i,j,:])
            result[i,j,:] = indices.spi(data[i,j,:],1,Distribution.gamma,1948,1948,2016,compute.Periodicity.monthly) 


In [6]:
np.save("spi1_half.npy",result)

In [7]:
np.save("lat_half.npy",lat)
np.save("lon_half.npy",lon)

# Africa Only

In [35]:
data = np.load('data.npy')
lat = np.load('lat.npy')
lon = np.load('lon.npy')

In [36]:
result = np.zeros(data.shape)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i,j,-1] < 0:
            result[i,j,:] = np.nan
        else:
            fuzzify(data[i,j,:])
            result[i,j,:] = indices.spi(data[i,j,:],3,Distribution.gamma,1946,1946,2016,compute.Periodicity.monthly) 

In [37]:
np.save("spi3.npy",result)

In [2]:
(2016-1891)*12

1500

In [17]:
result[9,0,11]

1.1646471321220377

In [75]:
result[21,5,:]

array([ 0.65474185,  1.32955962, -0.51363914, -0.38029055, -0.21054138,
       -1.01775135,  0.07868749, -0.16742007, -0.13010842,  1.95616933,
        0.3269889 , -0.42948132,  1.00763261, -1.02602837, -0.19910039,
       -0.32923873, -0.56300503, -0.40926219, -0.55129598, -0.50609702,
       -0.05613007, -0.5976439 ,  0.33774988, -0.53015901,  0.5127977 ,
        1.61021231, -0.6434171 ,  3.09      , -0.44114279, -2.45611227,
        0.82668115, -0.02576646, -0.16680235, -0.29505284,  0.79295105,
        0.72124683,  0.52995404, -0.73706542, -0.19872188, -0.45735626,
        1.49905661,  0.62054066, -0.19421866, -0.14462458,  0.23934405,
       -0.4205739 ,  0.06152377, -0.65127105,  0.85358976, -0.73092576,
       -0.62412632, -0.39526707,  0.56403734,  0.77999575,  1.11194536,
       -0.0480442 , -0.40009775, -0.25483382,  0.05527556, -0.63098226,
        0.65426101, -0.51496003, -0.33728654, -0.3138762 , -0.34669563,
       -0.35843299, -2.83113193, -0.38537322,  0.16507314, -0.44

In [70]:
data[21,5,:]

array([3.86038929e-01, 5.02786875e-01, 7.48184621e-02, 5.53643964e-02,
       3.97275984e-02, 3.25094769e-03, 4.47302731e-03, 2.82199332e-03,
       2.77701300e-03, 3.06699306e-01, 3.83414060e-01, 3.76157872e-02,
       5.45147479e-01, 6.12355769e-03, 1.26782313e-01, 6.45393133e-02,
       3.11939567e-02, 1.10999250e-03, 1.36506010e-03, 4.79852781e-03,
       2.08353414e-03, 9.36192274e-03, 3.79841387e-01, 4.37115133e-02,
       3.39900583e-01, 6.66266441e-01, 6.38658851e-02, 1.72778761e+00,
       3.42441760e-02, 2.27670162e-03, 6.89044886e-04, 6.46299624e-04,
       2.75241723e-03, 1.01125119e-02, 6.89628065e-01, 1.57625437e-01,
       3.38828534e-01, 6.10372517e-03, 1.25115201e-01, 4.74886373e-02,
       1.76535711e-01, 4.37497161e-03, 2.01513316e-03, 4.01881523e-03,
       1.43523887e-03, 1.02040721e-02, 2.52261400e-01, 3.37871313e-02,
       4.79497492e-01, 1.10845268e-02, 6.36566281e-02, 4.59552072e-02,
       8.74203444e-02, 4.50559659e-03, 3.70332412e-03, 4.40364750e-03,
      

# Based on Distribution

In [3]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        #st.beta,st.chi2,
        #st.expon,st.exponnorm,
        #st.genextreme,st.gamma,st.gengamma,
        #st.halfnorm,st.invgamma,st.invgauss,
        #st.norm,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,
        #st.uniform
        st.norm, st.gamma, st.pearson3
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
       # try:
        #    # Ignore warnings from data that can't be fit
         #   with warnings.catch_warnings():
          #      warnings.filterwarnings('ignore')

                # fit dist to data
        params = distribution.fit(data)

                # Separate parts of parameters
        arg = params[:-2]
        loc = params[-2]
        scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
        pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
        sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                #try:
                 #   if ax:
        #pd.Series(pdf, x).plot(ax=ax,legend = True, label = distribution.name)
                  #  end
               # except Exception:
                #    pass

                # identify if this distribution is better
        if best_sse > sse > 0:
            best_distribution = distribution
            best_params = params
            best_sse = sse

      #  except Exception:
     #       pass

    return (best_distribution.name, best_params)

In [21]:
data = np.load('data.npy')
lat = np.load('lat.npy')
lon = np.load('lon.npy')

In [22]:
result = np.zeros(data.shape)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i,j,-1] < 0:
            result[i,j,:] = np.nan
        else:
            fuzzify(data[i,j,:])
            for z in range(12):
                idx = np.arange(z,data.shape[2],12)
                d = data[i,j,idx]
                r = best_fit_distribution(d)
                if r[0] == 'gamma':
                    m = r[1][0]*r[1][2]
                else:
                    m = r[1][1]
                result[i,j,idx] = data[i,j,idx] - m 


KeyboardInterrupt: 

In [25]:
result

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.

In [22]:
idx = np.arange(0,841,12)

In [23]:
r = st.gamma.fit(data[20,20,idx])

In [24]:
data[20,20,idx].mean()

233.7844

In [20]:
r[0]*r[2]

221.37285156305083

In [21]:
idx

array([  1,  13,  25,  37,  49,  61,  73,  85,  97, 109, 121, 133, 145,
       157, 169, 181, 193, 205, 217, 229, 241, 253, 265, 277, 289, 301,
       313, 325, 337, 349, 361, 373, 385, 397, 409, 421, 433, 445, 457,
       469, 481, 493, 505, 517, 529, 541, 553, 565, 577, 589, 601, 613,
       625, 637, 649, 661, 673, 685, 697, 709, 721, 733, 745, 757, 769,
       781, 793, 805, 817, 829])